<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>CET Template Notebook</b> <br>
Contact author(s): <i>Author Name</i> <br>
Last verified to run: <i>yyyy-mm-dd</i> <br>
LSST Science Piplines version: Weekly <i>yyyy_xx</i> <br>
Container Size: <i>medium</i> <br>
Targeted learning level: <i>beginner</i> <br>

_In this template, text in italics are examples or instructions that should be: (a) removed if it is not applicable to the notebook; or (b) replaced with text that is appropriate for the notebook. But bold or regular text should appear pretty much as-is in all CET notebooks. For more information, see the [CET's Guidelines for Tutorial Notebooks](https://confluence.lsstcorp.org/pages/viewpage.action?pageId=168857070)._

_While developing, use the following code cell to check that the code conforms to standards, but then delete the cell and "Kernel --> Restart Kernel and Clear All Outputs" before saving and committing._

In [1]:
#%load_ext pycodestyle_magic
#%flake8_on
#import logging
#logging.getLogger("flake8").setLevel(logging.FATAL)#

_The six cells below are considered the extended header of the notebook. The first four will be used, verbatim, to create the table of notebook metadata in the README.md file for the repository._

**Description:** _Very brief description of notebook._

**Skills:** _Brief list of skills to match the README.md file for the repository._

**LSST Data Products:** _List the all of the types of LSST catalogs and images used._

**Packages:** _List the python packages used._ (_List the packages being taught first, e.g., afwDisplay for a notebook about displaying images. Then supporting packages, e.g., lsst.daf.butler for a notebook about displaying images. It is OK to leave out basic support packages like os or glob.)_

**Credit:**
_E.g., "Originally developed by" or "Based on notebooks developed by" and then people's names, including journal article or software release citations if appropriate._
Please consider acknowledging them if this notebook is used for the preparation of journal articles, software releases, or other notebooks.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-2.lsst.io">dp0-2.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

_Provide a light narrative about this notebook, e.g., "This notebook will teach the user..."._

_Cite or link to any external information or documentation, and cross-reference to other notebooks._

### 1.1 Package Imports

_All package imports should be done in the first code cell._

_Provide explanation or external links to package documentation, where appropriate._

_E.g., Numpy is a fundamental package for scientific computing with arrays in Python (<a href="https://numpy.org">numpy.org</a>)._

_Use code cell comments to describe the packages being imported._

In [2]:
# general python packages
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

# LSST package for TAP queries
from lsst.rsp import get_tap_service, retrieve_query

# LSST package for Butler queries
import lsst.daf.butler as dafButler

# LSST package for image display
import lsst.afw.display as afwDisplay

# general python packages
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

# LSST package for TAP queries
from lsst.rsp import get_tap_service, retrieve_query

# LSST package for Butler queries
import lsst.daf.butler as dafButler

# LSST package for image display
import lsst.afw.display as afwDisplay

from matplotlib import colormaps



In [3]:
service = get_tap_service("ssotap")


In [4]:
# Getting a list of ssObjectID for sources with perihelion distance < 1 AU and arc longer than 3000 days 
# to have sufficient phase angle coverage.
query0 = "SELECT mpc.ssObjectId, mpc.q, sso.ssObjectId, sso.arc " + \
        "FROM dp03_catalogs_10yr.MPCORB as mpc " + \
        "INNER JOIN dp03_catalogs_10yr.SSObject as sso " + \
        "ON mpc.ssObjectId = sso.ssObjectId " + \
        "WHERE sso.numObs > 300 AND sso.arc > 3000 "  + \
        "AND mpc.q < 1 ORDER by sso.arc"

df0 = service.search(query0).to_table()
df0

ssObjectId,q,ssObjectId2,arc
,AU,,d
int64,float64,int64,float32
5977535780727431144,0.6118754444868454,5977535780727431144,3003.859
4350915375550808373,0.36109415570675124,4350915375550808373,3008.7427
7114427843809346628,0.5180503801743852,7114427843809346628,3008.8723
169600859678073684,0.6063128526256948,169600859678073684,3011.9033
1015683152910208006,0.9687310781602868,1015683152910208006,3012.8008
5907446096447136661,0.5929850823219581,5907446096447136661,3019.7952
-2022279111318417234,0.5859395511579268,-2022279111318417234,3021.7915
-2974052514586114677,0.8945069913767757,-2974052514586114677,3022.8975


In [7]:
# Object from Meg's notebook: -9223369546614897710
ssObjId = df0['ssObjectId'][-3] 

# diaObjectId is the ID that links the individual measurements between DiaObject and SSSource,
# so you have to match on this id for unique measurement matching. 
# To then match this object to MPCORB and SSObject, you have
# to link on the SSObjectID (hwich is unique per solar system object)

query = "SELECT dia.diaSourceId, sss.diaSourceId, dia.mag, dia.ra, dia.dec," + \
        "sss.phaseAngle, sss.topocentricDist, " + \
        "sss.heliocentricDist, dia.midPointMjdTai " + \
        "FROM dp03_catalogs_10yr.DiaSource as dia " + \
        "JOIN dp03_catalogs_10yr.SSSource as sss " + \
        "ON dia.diaSourceId = sss.diaSourceId " + \
        "WHERE dia.ssObjectId = " + str(ssObjId) + ""


df = service.search(query).to_table()
df
ssObjId

2956589648411852100

### 1.2 Define Functions and Parameters

_If your notebook defines functions or parameters to use later or throughout, do it here in sub-section 1.2._

_It is OK to rename the subsection to be more specific to the notebook, and/or to use sub-sub-sections like "1.2.1 Define global cosmological parameter values" or "1.2.2 Define a function to make an image cutout"._

_It is OK to remove this sub-section if it is not being used._

## 2. Section Heading

_Use numbers for sections, sub-sections, and sub-sub-sections to enable referencing, e.g., "I'm having trouble with the second code cell in Section 2.3."_

_Use section titles that actively describe what is being done, e.g., "Create a color-magnitude diagram" instead of "Plot", so that the auto-generated table of contents is easy to navigate._

### 2.1 Section Sub-heading

#### 2.1.1 Section Sub-sub-heading